In [208]:
import numpy

class matrix_factorization():
    
    def __init__(self,R,a,b,it,k):
        self.R=R
        self.user_n, self.item_n=R.shape
        self.alpha=a
        self.beta=b
        self.iterations=it
        self.K=k
        
    def learn(self):
        self.b_users=numpy.zeros(self.user_n)
        self.b_items=numpy.zeros(self.item_n)
        self.b=numpy.mean(self.R[numpy.where(self.R!=0)])
        
        self.P=numpy.random.normal(scale=1./self.K,size=(self.user_n,self.K))
        self.Q=numpy.random.normal(scale=1./self.K,size=(self.item_n,self.K))
        
        self.samples=[]
        for i in range (self.user_n):
            for j in range (self.item_n):
                if (self.R[i][j])>0:
                    self.samples.append((i,j,self.R[i][j]))

        for i in range (self.iterations):
            numpy.random.shuffle(self.samples)
            self.sgd()
        self.c=self.complete()
            
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            self.b_users[i]+=self.alpha*(e-self.beta*self.b_users[i])
            self.b_items[j]+=self.alpha*(e-self.beta*self.b_items[j])
            
            self.P[i,:]+=self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:]+=self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
            
    def get_rating(self,i,j):
        return self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
    
    def complete(self):
        return self.P.dot(self.Q.T)+self.b+self.b_users[:,numpy.newaxis]+self.b_items[numpy.newaxis,:]
def Consensus(C):
    Cons=numpy.zeros((C[0].R.shape[0],C[0].R.shape[1],5))
    itc=numpy.nditer(Cons,flags=['multi_index'])
    for i in itc:
        y=(itc.multi_index[2]+1)
        for model in C:
            x=model.c[itc.multi_index[0]][itc.multi_index[1]]
            if(x==y):
                d=1
            else:
                d=1/abs(x-y)
            Cons[itc.multi_index[0]][itc.multi_index[1]][itc.multi_index[2]]+=d
    Cons=numpy.divide(Cons,len(C))
    for i in range (Cons.shape[0]):
        for j in range (Cons.shape[1]):
            norm=numpy.linalg.norm(Cons[i][j])
            Cons[i][j]=Cons[i][j]/norm
    return Cons

def kullback_leibler(C):
    K=numpy.zeros(C[0].R.shape)
    
    Cons=Consensus(C)
    print(Cons)
    
            
    itk=numpy.nditer(K,flags=['multi_index'],op_flags=['readwrite'])
    for x in itk:
        for model in C:
            p=[]
            for i in range(5):
                y=i+1
                if(model.c[itk.multi_index[0]][itk.multi_index[1]]==y):
                    d=1
                else:
                    d=1/abs(model.c[itk.multi_index[0]][itk.multi_index[1]]-y)
                d*=numpy.log(d/Cons[itk.multi_index[0]][itk.multi_index[1]][i])
                p.append(d)
            p=numpy.divide(p,numpy.linalg.norm(p))
            p=numpy.divide(p,len(C))
            s=p.sum()
        x+=s
    print(K)
    print(numpy.amax(K))
                
                

In [211]:
M = numpy.array([
    [5, 2, 0, 1],
    [1, 2, 5, 1],
    [0, 3, 0, 1],
    [1, 0, 3, 4],
    [5, 1, 0, 2],
])
print(M)

mf5 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf1 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf4= matrix_factorization(M, 0.1, 0.01, 20, 2)
mf6 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf2 = matrix_factorization(M, 0.1, 0.01, 20, 2)
mf3 = matrix_factorization(M, 0.1, 0.01, 20, 2)

mf1.learn()
mf2.learn()
mf3.learn()
mf4.learn()
mf5.learn()
mf6.learn()

kullback_leibler([mf2,mf1,mf3,mf4,mf5,mf6])


[[5 2 0 1]
 [1 2 5 1]
 [0 3 0 1]
 [1 0 3 4]
 [5 1 0 2]]
[[[0.00388246 0.00518695 0.00781175 0.01581677 0.9998234 ]
  [0.02422313 0.99946615 0.01853638 0.00969079 0.00656925]
  [0.07887563 0.4890571  0.79540995 0.21933888 0.27168963]
  [0.99911331 0.03655299 0.01628983 0.01052095 0.00777391]]

 [[0.99967443 0.02143101 0.01064809 0.00708435 0.00530796]
  [0.2528209  0.91040068 0.2895038  0.12839055 0.08340032]
  [0.02469112 0.03324747 0.05088413 0.10848054 0.99193131]
  [0.99153309 0.11656236 0.0448629  0.02860753 0.02107773]]

 [[0.32186069 0.55146874 0.1184308  0.1273457  0.7496967 ]
  [0.07067581 0.15557787 0.97703687 0.11217325 0.06013796]
  [0.51420622 0.37852725 0.28435416 0.3086184  0.64514067]
  [0.99986352 0.0142172  0.00654056 0.00425382 0.00315266]]

 [[0.99998769 0.00415831 0.00207846 0.00138555 0.00103914]
  [0.18848245 0.14803144 0.23903852 0.22399891 0.91391801]
  [0.00753664 0.0149     0.99970859 0.01563152 0.0077193 ]
  [0.0032155  0.00488383 0.01017839 0.99989171 0.0088